DNA methylation data preprocessing and analysis workflow based on the pipeline provided by Ehsan and collegues from Exeter University. 
This version will contain a mix of python native function if they are available and r integrations of functions that don't have a
python native version yet. Of these r integrated function a selection will be translated into python native functions if it is 
necessary for the continuation of the project.

Install the PyMethylProcess package from GitHub - https://github.com/Christensen-Lab-Dartmouth/PyMethylProcess/blob/master/pymethylprocess/MethylationDataTypes.py - because this has similar functionalities, based on the same functions, as those in the Exeter pipeline

In [1]:
pip install git+https://github.com/bodono/scs-python.git@bb45c69ce57b1fbb5ab23e02b30549a7e0b801e3 git+https://github.com/jlevy44/hypopt.git@af59fbed732f5377cda73fdf42f3d4981c2be3ce

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/bodono/scs-python.git (to revision bb45c69ce57b1fbb5ab23e02b30549a7e0b801e3) to c:\users\silke\appdata\local\temp\pip-req-build-x13x6rw9
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/bodono/scs-python.git 'C:\Users\Silke\AppData\Local\Temp\pip-req-build-x13x6rw9'
  Running command git checkout -q bb45c69ce57b1fbb5ab23e02b30549a7e0b801e3
  Running command git submodule update --init --recursive -q


  Cloning https://github.com/jlevy44/hypopt.git (to revision af59fbed732f5377cda73fdf42f3d4981c2be3ce) to c:\users\silke\appdata\local\temp\pip-req-build-rnn44w0l
Using legacy setup.py install for scs, since package 'wheel' is not installed.
Using legacy setup.py install for hypopt, since package 'wheel' is not installed.


  Running command git clone -q https://github.com/jlevy44/hypopt.git 'C:\Users\Silke\AppData\Local\Temp\pip-req-build-rnn44w0l'
You should consider upgrading via the 'C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\python.exe -m pip install --upgrade pip' command.


Install the methylsuite packages - https://pypi.org/project/methylsuite/ - because this contains functions that can read in raw data files 
rather than beta-matrices

In [2]:
pip install methylsuite

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Using legacy setup.py install for bs4, since package 'wheel' is not installed.
Using legacy setup.py install for toolshed, since package 'wheel' is not installed.
Using legacy setup.py install for interlap, since package 'wheel' is not installed.
    Running setup.py install for bs4: started
    Running setup.py install for bs4: finished with status 'done'
    Running setup.py install for toolshed: started
    Running setup.py install for toolshed: finished with status 'done'
    Running setup.py install for interlap: started
    Running setup.py install for interlap: finished with status 'done'


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\python.exe -m pip install --upgrade pip' command.


Importing the functionalities needed for the preprocessing and the analysis

In [2]:
from methylprep import make_pipeline, get_sample_sheet

In [3]:
from methylprep.files import create_sample_sheet, find_sample_sheet

In [4]:
import csv
import pandas as pd

Load the original series matrix

In [5]:
original_GSE66351_matrix = pd.read_table("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_RAW\\GSE66351_series_matrix_pheno.txt", header=None)

Rename the columns of the series matrix file so that they match the required column names for the import function from the methyprep package:
Has to include Sentrix_ID and Sentrix_position. The original file does not have these column names (GSE66351)

In [6]:
original_GSE66351_matrix_trans = original_GSE66351_matrix.transpose()
colum_names = ["Sample_Name", "GEO_Accession", "Sample_Status", "Sample_Submission_date", "Sample_last_update_data", "Sample_Type", "Channel_count", "Tissue_Source", "Organism", 
"Cell_Type", "Diagnosis", "Braak_stage", "Braak_region", "Age", "Sex", "Donor_id", "Sentrix_ID", "Sentrix_Position", "Molecule"]
original_GSE66351_matrix_trans.columns = colum_names

Remove the first row
Remove text from data frame cells (i.e Sex: in the cells of column Sex)
Set the index to the barcodes/idat file names

In [7]:
original_GSE66351_matrix_trans.drop([0], inplace=True)
original_GSE66351_matrix_trans.replace("^[^:]*:", "", regex=True, inplace=True)
original_GSE66351_matrix_trans["Sample_ID"] = original_GSE66351_matrix_trans[["GEO_Accession", "Sentrix_ID", "Sentrix_Position"]].agg("_".join, axis = 1)



Save the sample sheet as a .csv file so the methyprep function can deal with it

In [13]:
original_GSE66351_matrix_trans.to_pickle("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\Data\\GSE66351_RAW\\GSE66351_GPL13534_meta_data.pkl")
original_GSE66351_matrix_trans.to_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\Data\\GSE66351_RAW\\sample_sheet.csv")

Start with loading in the data, using functions from the methylsuite packages since these can deal with .idat files as input. 

In [56]:
from methylprep.download import convert_miniml
convert_miniml("GSE66351", data_dir = "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_RAW", merge=False)

INFO:methylprep.download.miniml:found 190 idat files; updated meta_data.
INFO:methylprep.download.miniml:Final samplesheet contains 190 rows and 16 columns


[WindowsPath('E:/Msc Systems Biology/MSB5000_Master_Thesis/Practical work/Data/GSE66351_RAW/GSE66351/GSE66351_family.xml')]

In [9]:
sample_sheet_GSE66351_at1 = find_sample_sheet("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\Data\\GSE66351_RAW")

Exception: Too many sample sheets in this directory. Move or rename redundant ones. Or specify the path to the one to use with --sample_sheet. (candidate files: [WindowsPath('E:/Msc Systems Biology/MSB5000_Master_Thesis/Practical work/Data/GSE66351_RAW/sample_sheet.csv'), WindowsPath('E:/Msc Systems Biology/MSB5000_Master_Thesis/Practical work/Data/GSE66351_RAW/GSE66351_GPL13534_samplesheet.csv')])

In [14]:
sample_sheet_GSE66351 = pd.read_pickle("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_RAW\\GSE66351_GPL13534_meta_data.pkl")

The file names of the idat files need to be changed for them to be read in using the methyprep package.

In [17]:
import os, glob, re

In [87]:
idat_folder = 'E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_python_format\\'
idat_files_only = idat_folder + "*.idat"
old_filenames = glob.glob(idat_files_only)


Create the list of new filenames

In [95]:
file_name = old_filenames[1]
new_name = re.sub("(GSM\d[1-9]\d[1-9]\d[1-9][1-9]_)", " ", file_name)
print(new_name)

E:\Msc Systems Biology\MSB5000_Master_Thesis\Practical work\Data\GSE66351_python_format\GSM2808880_8918692108_R03C01_Red.idat


In [67]:
pattern_to_remove = "(GSM\d[0-9]\d[0-9]\d[0-9][0-9]_)"
for file_name in old_filenames:
    old_name = file_name
    new_name = re.sub("(GSM\d[0-9]\d[0-9]\d[0-9][0-9]_)", "", file_name)
    final_name = new_name
    os.replace(old_name, final_name)

new_filenames = glob.glob(idat_files_only) # just to check if the renaming went the way it should have

Reading in the data (finally)

In [75]:
data = make_pipeline("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_python_format" , sample_sheet_filepath = "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\GSE66351_python_format\\sample_sheet.csv" , sample_name = sample_names, steps= None, exports= None, estimator= "beta")

INFO:methylprep.processing.pipeline:Running pipeline in: E:\Msc Systems Biology\MSB5000_Master_Thesis\Practical work\Data\GSE66351_python_format
INFO:methylprep.processing.pipeline:Found 17 additional fields in sample_sheet:
Sample_ID | GEO_Accession | Sample_Status | Sample_Submission_date | Sample_last_update_data | Channel_count | Tissue_Source | Organism | Cell_Type | Diagnosis | Braak_stage | Braak_region | Age | Sex | Donor_id | Molecule | Sample_ID.1 --> Sample_ID1
Reading IDATs:   0%|          | 0/190 [00:00<?, ?it/s]


FileNotFoundError: No files in E:\Msc Systems Biology\MSB5000_Master_Thesis\Practical work\Data\GSE66351_python_format (or sub-folders) match this sample id: 8918692108_R01C02_Grn.idat OR 8918692108_R01C02_Grn.idat

In [62]:
sample_sheet = get_sample_sheet("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\Data\\GSE66351_python_format")

In [80]:
samples = sample_sheet.get_samples()
samples[1].get_filepath('idat')
"E:\Msc Systems Biology\MSB5000_Master_Thesis\Practical work\Data\GSE66351_python_format\8918692108_R01C01_Red.idat"

FileNotFoundError: No files in E:\Msc Systems Biology\MSB5000_Master_Thesis\Practical work\Data\GSE66351_python_format (or sub-folders) match this sample id: 8918692108_R01C01.idat OR 8918692108_R01C01.idat

In [85]:
for _index, row in sample_sheet.__data_frame.iterrows():
            sentrix_id = row['sentrix_ID'].strip()
            sentrix_position = row['sentrix_position'].strip()

AttributeError: 'SampleSheet' object has no attribute '__data_frame'